# 1. Naver Sentiment Movie Corpus 다운로드

In [1]:
! wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
! wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

--2024-01-20 08:19:41--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings_train.txt’

ratings_train.txt   100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2024-01-20 08:19:41 (113 MB/s) - ‘ratings_train.txt’ saved [14628807/14628807]

--2024-01-20 08:19:41--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving

# 2. Vocab 로드
: Sentencepiece를 활용해 만들어 놓은 vocab을 로드

In [4]:
! pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00


In [5]:
# vocab loading
import sentencepiece as spm
vocab_file = "kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

# 3. 전처리
: 다운로드한 데이터를 vocab으로 미리 tokenize해서 json형태로 저장
(tokenize를 미리하지 않고 training시에 할 경우 처리시간이 매번 소요 되므로 이를 효과적으로 줄이기 위함)

In [10]:
import pandas as pd
import json
def prepare_train(vocab, infile, outfile):
    df = pd.read_csv(infile, sep="\t", engine="python")
    with open(outfile, "w") as f:
        for index, row in df.iterrows(): # 각 행을 순회
            document = row["document"]
            if type(document) != str:
                continue
            instance = { "id": row["id"], "doc": vocab.encode_as_pieces(document), "label": row["label"] }
                                                 # vocab.encode_as_pieces(document)를 사용하여 단어 사전에 따라 문서를 부분으로 나눔
            # instance라는 딕셔너리를 생성하여 "id", "doc", "label" 키를 가지고 있음
            f.write(json.dumps(instance))
            f.write("\n")
            #  JSON 형식으로 변환하여 출력 파일에 기록. 마지막에 줄 바꿈 문자("\n")을 추가하여 각 인스턴스를 새로운 줄에 저장

In [11]:
prepare_train(vocab, "ratings_train.txt", "ratings_train.json")
prepare_train(vocab, "ratings_test.txt", "ratings_test.json")